In [ ]:
# wget https://huggingface.co/hkchengrex/MMAudio/resolve/main/ext_weights/synchformer_state_dict.pth?download=true

In [1]:
from mmaudio.ext.synchformer import Synchformer

synchformer = Synchformer()
synchformer.eval()
synchformer.to('cuda')
print("-")

-


In [2]:
import torch

ckpt_path = './synchformer_state_dict.pth'
synchformer.load_state_dict(torch.load(ckpt_path, weights_only=True, map_location='cpu'))

<All keys matched successfully>

In [5]:
from einops import rearrange

@torch.inference_mode()
def encode_video_with_sync(x: torch.Tensor, batch_size: int = -1) -> torch.Tensor:
    # x: (B, T, C, H, W) H/W: 384
    b, t, c, h, w = x.shape
    assert c == 3 and h == 224 and w == 224

    # partition the video
    segment_size = 16
    step_size = 8
    num_segments = (t - segment_size) // step_size + 1
    segments = []
    for i in range(num_segments):
        segments.append(x[:, i * step_size:i * step_size + segment_size])
    x = torch.stack(segments, dim=1)  # (B, S, T, C, H, W)

    outputs = []
    if batch_size < 0:
        batch_size = b
    x = rearrange(x, 'b s t c h w -> (b s) 1 t c h w')
    for i in range(0, b * num_segments, batch_size):
        outputs.append(synchformer(x[i:i + batch_size]))
    x = torch.cat(outputs, dim=0)
    x = rearrange(x, '(b s) 1 t d -> b (s t) d', b=b)
    return x

sample_video = torch.randn((2, 24*2, 3, 224, 224)).to('cuda')
out = encode_video_with_sync(sample_video)
print(out.shape)

torch.Size([2, 40, 768])


In [5]:
import torch.nn.functional as F
import torch

xs = torch.randn((2, 768, 40))
ds = F.interpolate(xs, size=43, mode='nearest-exact')

In [6]:
ds.shape

torch.Size([2, 768, 43])